In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
import chess

import numpy as np

In [11]:
training_generator = DataLoader(128, "train")
val_generator = DataLoader(128, "val")

In [15]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=1.984375)

In [13]:
halfAcc = Dense(256, clipped_relu)
i1 = tf.keras.Input((64*64*12,))
i2 = tf.keras.Input((64*64*12,))
acc1 = halfAcc(i1)
acc2 = halfAcc(i2)
layerStack = tf.keras.layers.Concatenate()([acc1, acc2])

h1 = Dense(32, clipped_relu)
h2 = Dense(32, clipped_relu)
out = Dense(1, 'sigmoid')

layerStack = out(h2(h1(layerStack)))

model = tf.keras.Model(inputs=[i1, i2], outputs=layerStack)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

In [14]:
#921/921 [==============================] - 24s 26ms/step - loss: 0.1499 - mae: 0.3252 - val_loss: 0.1661 - val_mae: 0.3544
#921/921 [==============================] - 25s 27ms/step - loss: 0.1361 - mae: 0.3087 - val_loss: 0.1685 - val_mae: 0.3563

model.fit(training_generator, validation_data=val_generator, epochs=1)

2981/2981 [==============================] - 76s 25ms/step - loss: 0.3785 - mae: 0.4697 - val_loss: 0.3862 - val_mae: 0.4566


In [9]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

[0.07652974] 1.0 2r2rk1/1pq1bppp/p1np1n2/4pP2/5P2/P1N1B3/1PP1B1PP/R2QR2K b - - 2 14
[0.05268109] 1.0 4rrk1/3n2p1/ppq1p2p/2p2p2/3Pn3/P1PBP3/1BQ2PPP/R4RK1 w - - 4 20
[-0.06599802] 1.0 8/8/p3rk2/2P2p2/2R5/P7/1P1K4/8 w - - 2 42
[-0.43313777] 1.0 3r2k1/p2r2p1/Q2p3p/p1pPpp2/2P5/1RPqP3/5PPP/1R5K w - - 0 27
[-0.00580215] 0.0 6R1/8/5b2/3k4/1K6/5r2/8/8 b - - 52 101
[-0.08939898] 1.0 r6k/1p3p2/1p2b2p/1P5N/4P3/n2B1K2/3r2PP/R6R w - - 3 26
[0.33327663] 1.0 2rr2k1/pqnB2pp/1p3p2/3p4/5Q2/1P4P1/PB2PP1P/2RR2K1 b - - 0 26
[-0.3123998] 0.0 8/4rkp1/1p5p/3R1p2/P7/1P5P/6P1/3Qq2K w - - 3 49
[-0.46692902] -1.0 6k1/3b2bp/3P2p1/5p2/8/4B2P/3Rq1P1/r4NK1 w - - 0 51
[0.32468903] 1.0 1r6/4k1p1/2b1p3/r2p1pP1/P1p2P2/2N1P1K1/RP5R/8 w - - 43 78
[-0.08458406] -1.0 r2r4/1b2bp1k/p1nqp2p/5p2/N1Pp4/P4N2/1PQ1BPPP/3R1RK1 w - - 1 23
[0.09309387] 1.0 rnbqk2r/ppp2ppp/4p3/3n4/1b1P4/2N5/PPQ1PPPP/R1B1KBNR w KQkq - 0 6
[-0.15443814] 0.0 8/1Q4k1/4q3/4P1p1/8/1p3P2/6K1/8 b - - 10 80
[0.4283594] 1.0 5k2/8/6K1/6P1/8/8/8/8 w - - 3 67
[0.1918

In [5]:
import time
model.save("models/model-{}".format(int(time.time())))

2022-03-26 19:52:09.820283: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model-1648338729/assets
